In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### Read

In [ ]:
temp_zuz = pd.read_csv('sample_data/temp_zuz.csv', delimiter=';')
temp_zuz['Czas'] = pd.to_datetime(temp_zuz['Czas'])

In [ ]:
df = pd.read_csv('sample_data/avg_from_2020_10_01_00_00_00_to_2020_10_01_23_59_00')
df['czas'] = pd.to_datetime(df['czas'])

In [ ]:
feature_desc_df = pd.read_csv('feature_desc.csv', index_col='name')
def feature_desc(name):
    if type(name)==str:
        return feature_desc_df.loc[name]['desc']
    else:
        return feature_desc_df.loc[name]['desc'].values

### Group

In [ ]:
TIR = df.columns[df.columns.str.contains('tir')].values
FIR = df.columns[df.columns.str.contains('fir')].values
TIX1 = df.columns[df.columns.str.contains('001tix')].values
UXM = df.columns[df.columns.str.contains('uxm')].values
TXI = df.columns[df.columns.str.contains('txi')].values
TIX37 = df.columns[df.columns.str.contains('037tix')].values
FCX = df.columns[df.columns.str.contains('fcx')].values
PR = df.columns[df.columns.str.contains('pr')].values
NIR = df.columns[df.columns.str.contains('nir')].values

group_names = {
    'TIR':'WODY POWROTNE KOLEKTORÓW [°C]',
    'FIR':'WODY CHŁODZĄCE KOLEKTORÓW [°C]',
    'TIX1':'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]',
    'UXM':'WENT. ODCZ_ZAD OBROTÓW [%]',
    'TXI':'TEMP. NA KOL. [°C]',
    'TIX37':'TEMP. WODY ZASIL.OBIEG PZ [°C]',
    'FCX':'REG [Mg/h]',
    'PR':'Prob i prazonka [%]',
    'NIR':'SUMARYCZNA MOC CIEPLNA ODEBRANA - CAŁKOWITA [MW]'
}   
group_names

### Distributions

In [ ]:
def plot_histograms(columns, ncols=3):
    n = len(columns)
    if n<ncols:
        ncols=n
        nrows=1
    else:
        nrows = int(np.ceil(n/ncols))
        
    f, axes = plt.subplots(figsize=(14, (8*nrows)//ncols), nrows=nrows, ncols=ncols)
    plt.subplots_adjust(wspace=0.2, hspace=0.6)
    for col, ax in zip(columns, axes.flatten()):
        sns.histplot(df[col], ax=ax, kde=True)
        ax.set_title(feature_desc(col), {'fontsize':10})

In [ ]:
plot_histograms(TIR)

In [ ]:
plot_histograms(FIR)

In [ ]:
plot_histograms(TIX1)

In [ ]:
plot_histograms(UXM)

In [ ]:
plot_histograms(TXI)

In [ ]:
plot_histograms(TIX37)

In [ ]:
plot_histograms(FCX, ncols=2)

In [ ]:
plot_histograms(np.append(PR, NIR))

### Correlations

In [ ]:
def corr_plots(*dfs, titles=None, ncols=2, size=14):
    n = len(dfs)
    if n<ncols:
        ncols=n
        nrows=1
    else:
        nrows = int(np.ceil(n/ncols))     
    
    f, axes = plt.subplots(figsize=(size, (size*nrows)//ncols), nrows=nrows, ncols=ncols, squeeze=False)
    plt.subplots_adjust(wspace=0.4, hspace=0.2)
    
    for df, ax, i in zip(dfs, axes.flatten(), range(n)):
        corr = df.corr()
        cmap = sns.diverging_palette(230, 20, as_cmap=True)
        sns.heatmap(corr, cmap=cmap, center=0,
                square=True, linewidths=.5, ax=ax, annot=(len(df.columns)*ncols<25))
        if titles:
            ax.set_title(titles[i])

In [ ]:
corr_plots(df, titles=["All features"])

In [ ]:
corr_plots(df[TIR], 
           df[FIR], 
           titles=["WODY POWROTNE KOLEKTORÓW [°C]", 
                   "WODY CHŁODZĄCE KOLEKTORÓW [°C]"]
          )

In [ ]:
corr_plots(df[TIX1],
           size=14,
           titles=[group_names['TIX1']]
          )

In [ ]:
corr_plots(df[np.concatenate([UXM, TXI, TIX37])],
           df[np.concatenate([FCX, PR, NIR])],
          titles=[group_names['UXM'] + '\n' + group_names['TXI'] + '\n' + group_names['TIX37'],
                  group_names['FCX'] + '\n' + group_names['PR'] + '\n' + group_names['NIR']]
                  )

### PCA

In [ ]:
scaler = StandardScaler()
data  = scaler.fit_transform(df[df.columns[1:]])

pca = PCA(n_components=3)
components = pca.fit_transform(data)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2,
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'},
    height=500
)
fig.show()

### Simple Hourly Aggregates

In [ ]:
df_agg = df[:]
df_agg['czas'] = df_agg['czas'].dt.floor("H")
df_agg.groupby('czas')['001fcx00211.pv'].mean()
# tbc